In [529]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from termcolor import colored
import requests
from bs4 import BeautifulSoup
from collections import Counter
pc = ['#FFB6C1', '#FFA07A', '#FFD700', '#98FB98', '#DDA0DD', '#87CEEB', '#F0E68C', '#FFE4E1', '#98FB98', '#FF69B4']


In [530]:
df = pd.read_csv('India_odi_bat.csv')

In [531]:
df.head()

,Runs,Mins,BF,4s,6s,SR,Pos,Dismissal,Inns,Unnamed: 9,Opposition,Ground,Start Date,Unnamed: 13,Player
0,101*,195,121,10,0,83.47,3,not out,1,NaN,v South Africa,Eden Gardens,5 Nov 2023,ODI # 4694,Virat
1,88,139,94,11,0,93.61,3,caught,1,NaN,v Sri Lanka,Wankhede,2 Nov 2023,ODI # 4690,Virat
2,0,12,9,0,0,0.00,3,caught,1,NaN,v England,Lucknow,29 Oct 2023,ODI # 4686,Virat
3,95,154,104,8,2,91.34,3,caught,2,NaN,v New Zealand,Dharamsala,22 Oct 2023,ODI # 4678,Virat
4,103*,113,97,6,4,106.18,3,not out,2,NaN,v Bangladesh,Pune,19 Oct 2023,ODI # 4674,Virat


In [532]:
df.pop(df.columns[13])

0       ODI # 4694
1       ODI # 4690
2       ODI # 4686
3       ODI # 4678
4       ODI # 4674
           ...    
1603    ODI # 4651
1604    ODI # 4654
1605    ODI # 4666
1606    ODI # 4669
1607    ODI # 4674
Name: Unnamed: 13, Length: 1608, dtype: object

In [533]:
df.pop(df.columns[9])

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
1603   NaN
1604   NaN
1605   NaN
1606   NaN
1607   NaN
Name: Unnamed: 9, Length: 1608, dtype: float64

In [534]:
df.pop(df.columns[7])

0       not out
1        caught
2        caught
3        caught
4       not out
         ...   
1603          -
1604          -
1605          -
1606          -
1607          -
Name: Dismissal, Length: 1608, dtype: object

In [535]:
df.head()

,Runs,Mins,BF,4s,6s,SR,Pos,Inns,Opposition,Ground,Start Date,Player
0,101*,195,121,10,0,83.47,3,1,v South Africa,Eden Gardens,5 Nov 2023,Virat
1,88,139,94,11,0,93.61,3,1,v Sri Lanka,Wankhede,2 Nov 2023,Virat
2,0,12,9,0,0,0.00,3,1,v England,Lucknow,29 Oct 2023,Virat
3,95,154,104,8,2,91.34,3,2,v New Zealand,Dharamsala,22 Oct 2023,Virat
4,103*,113,97,6,4,106.18,3,2,v Bangladesh,Pune,19 Oct 2023,Virat


In [536]:
df.shape

(1608, 12)

In [537]:
df = df[df['Runs'] != 'DNB']
df = df[df['Runs'] != 'TDNB']

In [538]:
df.shape

(1181, 12)

In [539]:
indian_stadiums = ['Eden Gardens', 'Wankhede', 'Lucknow', 'Dharamsala','Pune','Ahmedabad','Delhi','Chennai','Rajkot'\
'Visakhapatnam','Indore','Hyderabad','Thiruvananthapuram','Raipur','Guwahati','Bengaluru','Cuttack','Nagpur'\
                   ,'Ranchi','Mohali']
df['Ground'] = df['Ground'].apply(lambda x: 1 if x in indian_stadiums else 0)

In [540]:
df['Start Date'] = pd.to_datetime(df['Start Date'], format='%d %b %Y').dt.year
df['Opposition'] = df['Opposition'].str[2:]

In [541]:
df['Mins'] = pd.to_numeric(df['Mins'], errors='coerce')
df['BF'] = pd.to_numeric(df['BF'], errors='coerce')
df['4s'] = pd.to_numeric(df['4s'], errors='coerce')
df['6s'] = pd.to_numeric(df['6s'], errors='coerce')
df['SR'] = pd.to_numeric(df['SR'], errors='coerce')
df['Pos'] = pd.to_numeric(df['Pos'], errors='coerce')
df['Inns'] = pd.to_numeric(df['Inns'], errors='coerce')
df['NO'] = df['Runs'].str.endswith('*').astype(int)
df['Runs'] = df['Runs'].str.replace('*', '')
df['Runs'] = pd.to_numeric(df['Runs'], errors='coerce')

<ipython-input-541-c49f0468a10c>:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Runs'] = df['Runs'].str.replace('*', '')


In [542]:
df.head()

,Runs,Mins,BF,4s,6s,SR,Pos,Inns,Opposition,Ground,Start Date,Player,NO
0,101,195.0,121,10,0,83.47,3,1,South Africa,1,2023,Virat,1
1,88,139.0,94,11,0,93.61,3,1,Sri Lanka,1,2023,Virat,0
2,0,12.0,9,0,0,0.00,3,1,England,1,2023,Virat,0
3,95,154.0,104,8,2,91.34,3,2,New Zealand,1,2023,Virat,0
4,103,113.0,97,6,4,106.18,3,2,Bangladesh,1,2023,Virat,1


In [543]:
df.describe()

,Runs,Mins,BF,4s,6s,SR,Pos,Inns,Ground,Start Date,NO
count,1181.000000,1028.000000,1181.000000,1181.000000,1181.000000,1164.000000,1181.000000,1181.000000,1181.000000,1181.000000,1181.000000
mean,33.180356,51.357004,35.664691,3.007621,0.705334,82.630086,4.807790,1.500423,0.340390,2017.041490,0.225233
std,37.932194,50.451920,35.282459,3.899471,1.447223,50.302241,2.785764,0.500212,0.474041,4.624086,0.417913
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,2007.000000,0.000000
25%,5.000000,13.000000,8.000000,0.000000,0.000000,55.302500,3.000000,1.000000,0.000000,2013.000000,0.000000
50%,18.000000,32.000000,22.000000,2.000000,0.000000,81.750000,4.000000,2.000000,0.000000,2017.000000,0.000000
75%,51.000000,78.250000,55.000000,5.000000,1.000000,104.410000,7.000000,2.000000,1.000000,2022.000000,0.000000
max,264.000000,228.000000,173.000000,33.000000,16.000000,600.000000,11.000000,2.000000,1.000000,2023.000000,1.000000


In [544]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(list(df['Opposition']))
df['Opposition'] = df['Opposition'].apply(lambda x: le.transform([x])[0])

In [545]:
df.describe()

,Runs,Mins,BF,4s,6s,SR,Pos,Inns,Opposition,Ground,Start Date,NO
count,1181.000000,1028.000000,1181.000000,1181.000000,1181.000000,1164.000000,1181.000000,1181.000000,1181.000000,1181.000000,1181.000000,1181.000000
mean,33.180356,51.357004,35.664691,3.007621,0.705334,82.630086,4.807790,1.500423,7.232007,0.340390,2017.041490,0.225233
std,37.932194,50.451920,35.282459,3.899471,1.447223,50.302241,2.785764,0.500212,4.643606,0.474041,4.624086,0.417913
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,2007.000000,0.000000
25%,5.000000,13.000000,8.000000,0.000000,0.000000,55.302500,3.000000,1.000000,2.000000,0.000000,2013.000000,0.000000
50%,18.000000,32.000000,22.000000,2.000000,0.000000,81.750000,4.000000,2.000000,8.000000,0.000000,2017.000000,0.000000
75%,51.000000,78.250000,55.000000,5.000000,1.000000,104.410000,7.000000,2.000000,11.000000,1.000000,2022.000000,0.000000
max,264.000000,228.000000,173.000000,33.000000,16.000000,600.000000,11.000000,2.000000,14.000000,1.000000,2023.000000,1.000000


In [546]:
import numpy as np
conditions = [
    (df['Runs'] >= 100),
    (df['Runs'] >= 50)
]
values = [2, 1]
df['milestone'] = np.select(conditions, values, default=0)

In [547]:
df.head()

,Runs,Mins,BF,4s,6s,SR,Pos,Inns,Opposition,Ground,Start Date,Player,NO,milestone
0,101,195.0,121,10,0,83.47,3,1,10,1,2023,Virat,1,2
1,88,139.0,94,11,0,93.61,3,1,11,1,2023,Virat,0,1
2,0,12.0,9,0,0,0.00,3,1,3,1,2023,Virat,0,0
3,95,154.0,104,8,2,91.34,3,2,8,1,2023,Virat,0,1
4,103,113.0,97,6,4,106.18,3,2,2,1,2023,Virat,1,2


Algorithm for batsman :
0.30*run_scored+0.05*notout_innigs+
0.20*bat_avg + 0.15*bat_sr+0.15*milestone_reaching_ability+ 0.10*no_of_4’s_6’s+0.05*high_score-0.05*no_of_zeroes
β_pos_score =max (α_batsmen_score_at_each_position)
4: γ_inningwise_score = 0.40*α_batsmen_score_first_inning+
0.60*α_batsmen_score_seocnd_inning
5: x_venue = 0.35*α_batsmen_score_home_matches+0.65*
α_batsmen_score_away_matches
6: y_opponent = 0.70*α_batsmen_score_strong_opponent+0.30*
α_batsmen_score_weak_opponent
7: w_yearwise = 0.20*α_batsmen_score_current_year+0.80*
α_batsmen_score_last_five_year
8:batting_score = 0.25*α_batsmen_score+0.10*β_pos_score+0.15*γ_inningwise_score+0.15*x_venue +0.15*
y_opponent+0.20*w_yearwise


In [548]:
def coun(data):
  if data.shape[0] == 0:
    return 0
  else :
    n= 1/data.shape[0]
  ans = (data['Runs'].sum()*0.3) + (0.05*data[data['NO'] == 1].shape[0]) +(0.20*data['Runs'].mean())\
  +(data['SR'].mean()*0.15) + (15*(data[data['milestone'] == 1].shape[0]+data[data['milestone'] == 2].shape[0])*n) +\
  (0.10 * (data['4s'].sum()+data['6s'].sum())) + (0.05 * data['Runs'].max()) - (0.05 * data[data['Runs'] == 0].shape[0])
  return ans

In [549]:
def count(df):
  alpha_batsmen_score = coun(df)
  unp = df['Pos'].unique()
  beta_batsmen_score = 0
  for i in range(len(unp)):
    ans = coun(df[df['Pos'] == unp[i]])
    beta_batsmen_score = max (beta_batsmen_score,ans)
  gemma_batsmen_score = (0.40*coun(df[df['Inns'] == 1])) + (0.60*coun(df[df['Inns'] == 2]))
  x_venue = (0.35*coun(df[df['Ground'] == 1])) + (0.65*coun(df[df['Ground'] == 0]))
  y_opponent = (0.70*coun(df[df['Opposition'] == 3])) + (0.30*coun(df[df['Opposition'] == 7]))
  w_yearwise =(0.80*coun(df[(df['Start Date'] >= 2018) & (df['Start Date'] <= 2022)])) \
  + (0.20*coun(df[df['Start Date'] == 2023]))
  batting_score = (0.25*alpha_batsmen_score) + (0.10*beta_batsmen_score) +\
  (0.15*gemma_batsmen_score) + (0.15*x_venue) + (0.15*y_opponent) + (0.20*w_yearwise)
  return batting_score

In [550]:
ans = df[df['Player']=='Kuldeep']
count(ans)

42.14515846684529

In [551]:
df2 = pd.read_csv('India_odi_bow.csv')

In [552]:
df2.head()

,Overs,Mdns,Runs,Wkts,Econ,Pos,Inns,Unnamed: 7,Opposition,Ground,Start Date,Unnamed: 11,Player
0,DNB,-,-,-,-,-,2,NaN,v Sri Lanka,Dambulla,18 Aug 2008,ODI # 2742,Virat
1,DNB,-,-,-,-,-,1,NaN,v Sri Lanka,Dambulla,20 Aug 2008,ODI # 2745,Virat
2,DNB,-,-,-,-,-,2,NaN,v Sri Lanka,Colombo (RPS),24 Aug 2008,ODI # 2750,Virat
3,DNB,-,-,-,-,-,2,NaN,v Sri Lanka,Colombo (RPS),27 Aug 2008,ODI # 2755,Virat
4,DNB,-,-,-,-,-,1,NaN,v Sri Lanka,Colombo (RPS),29 Aug 2008,ODI # 2756,Virat


In [553]:
df2.pop(df2.columns[11])

0       ODI # 2742
1       ODI # 2745
2       ODI # 2750
3       ODI # 2755
4       ODI # 2756
           ...    
1603    ODI # 4651
1604    ODI # 4654
1605    ODI # 4666
1606    ODI # 4669
1607    ODI # 4674
Name: Unnamed: 11, Length: 1608, dtype: object

In [554]:
df2.pop(df2.columns[7])

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
1603   NaN
1604   NaN
1605   NaN
1606   NaN
1607   NaN
Name: Unnamed: 7, Length: 1608, dtype: float64

In [555]:
df2 = df2[df2['Overs'] != 'DNB']
df2 = df2[df2['Overs'] != 'TDNB']

In [556]:
df2['Overs'] = pd.to_numeric(df2['Overs'], errors='coerce')
df2['Mdns'] = pd.to_numeric(df2['Mdns'], errors='coerce')
df2['Runs'] = pd.to_numeric(df2['Runs'], errors='coerce')
df2['Wkts'] = pd.to_numeric(df2['Wkts'], errors='coerce')
df2['Econ'] = pd.to_numeric(df2['Econ'], errors='coerce')
df2['Pos'] = pd.to_numeric(df2['Pos'], errors='coerce')
df2['Inns'] = pd.to_numeric(df2['Inns'], errors='coerce')
df2['Start Date'] = pd.to_datetime(df2['Start Date'], format='%d %b %Y').dt.year

In [557]:
df2.describe()

,Overs,Mdns,Runs,Wkts,Econ,Pos,Inns,Start Date
count,899.000000,899.000000,899.000000,899.000000,899.000000,899.000000,899.000000,899.000000
mean,7.675195,0.340378,39.161290,1.314794,5.247920,3.875417,1.417130,2016.955506
std,2.785785,0.631261,17.643071,1.235118,1.923812,1.630051,0.493359,4.302615
min,0.100000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,2007.000000
25%,6.000000,0.000000,27.000000,0.000000,4.000000,3.000000,1.000000,2013.000000
50%,9.000000,0.000000,39.000000,1.000000,5.000000,4.000000,1.000000,2017.000000
75%,10.000000,1.000000,51.000000,2.000000,6.200000,5.000000,2.000000,2021.000000
max,10.000000,4.000000,87.000000,6.000000,18.000000,9.000000,2.000000,2023.000000


In [558]:
df2.isnull().sum()

Overs         0
Mdns          0
Runs          0
Wkts          0
Econ          0
Pos           0
Inns          0
Opposition    0
Ground        0
Start Date    0
Player        0
dtype: int64

In [559]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(list(df2['Opposition']))
df2['Opposition'] = df2['Opposition'].apply(lambda x: le.transform([x])[0])

In [560]:
indian_stadiums = ['Eden Gardens', 'Wankhede', 'Lucknow', 'Dharamsala','Pune','Ahmedabad','Delhi','Chennai','Rajkot'\
'Visakhapatnam','Indore','Hyderabad','Thiruvananthapuram','Raipur','Guwahati','Bengaluru','Cuttack','Nagpur'\
                   ,'Ranchi','Mohali']
df2['Ground'] = df2['Ground'].apply(lambda x: 1 if x in indian_stadiums else 0)

In [561]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 899 entries, 6 to 1607
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Overs       899 non-null    float64
 1   Mdns        899 non-null    int64  
 2   Runs        899 non-null    int64  
 3   Wkts        899 non-null    int64  
 4   Econ        899 non-null    float64
 5   Pos         899 non-null    int64  
 6   Inns        899 non-null    int64  
 7   Opposition  899 non-null    int64  
 8   Ground      899 non-null    int64  
 9   Start Date  899 non-null    int64  
 10  Player      899 non-null    object 
dtypes: float64(2), int64(8), object(1)
memory usage: 84.3+ KB


In [562]:
def bowscore(data):
  if data.shape[0] == 0 :
    return 0;
  cw = 1
  if data['Wkts'].sum() != 0 :
    cw = 1/data['Wkts'].sum()
  else :
    cw = 0
  ans = (data['Wkts'].sum()*0.3) + (0.20*data['Runs'].sum()*cw) + \
  (0.10*((((data['Overs'] * 10) % 10).astype(int))+ (data['Overs'].astype(int) * 6)).sum()) + \
  (0.15*data['Runs'].sum()/data['Overs'].sum()) + (0.10*(data[data['Wkts']==4].shape[0]+data[data['Wkts']==5].shape[0])) +\
  (0.05*data['Wkts'].max() + (0.10*data['Mdns'].sum()) )
  return ans

In [563]:
def count2(data):
  if data.shape[0] == 0:
    return 0
  alpha_bow_score = bowscore(data)
  beta_bow_score = (0.40*bowscore(data[data['Inns'] == 1])) + (0.60*bowscore(data[data['Inns'] == 2]))
  x_venue = (0.40*bowscore(data[data['Ground'] == 1])) + (0.60*bowscore(data[data['Ground'] == 0]))
  y_opponent = (0.80*bowscore(data[data['Opposition'] == 3])) + (0.20*bowscore(data[data['Opposition'] == 7]))
  w_yearwise =(0.80*bowscore(data[(data['Start Date'] >= 2018) & (data['Start Date'] <= 2022)]))
  + (0.20*bowscore(data[data['Start Date'] == 2023]))
  bow_score = (0.30*alpha_bow_score) + (0.20*beta_bow_score) +\
  + (0.15*0) + (0.20*0) + (0.15*0)
  return bow_score

In [564]:
count2(df2[df2['Player']=='Ashwin'])

273.13007034491045

In [592]:
def batall(name):
  batal = (count2(df2[df2['Player']==name])*0.30) + (0.50*count(df[df['Player']==name])) + (0.20*(count(df[df['Player']==name])-count2(df2[df2['Player']==name])))
  return batal

In [588]:
def bowall(name):
  bowal = (count2(df2[df2['Player']==name])*0.45) + (0.35*count(df[df['Player']==name])) + (0.20*(count(df[df['Player']==name])-count2(df2[df2['Player']==name])))
  return bowal

In [567]:
df3 = pd.read_csv('India_odi_f.csv')

In [568]:
df3.head()

,Dis,Ct,St,Ct Wk,Ct Fi,Inns,Unnamed: 6,Opposition,Ground,Start Date,Unnamed: 10,Player
0,0,0,0,0,0,2,NaN,v Sri Lanka,Dambulla,18 Aug 2008,ODI # 2742,Virat
1,3,3,0,0,3,1,NaN,v Sri Lanka,Dambulla,20 Aug 2008,ODI # 2745,Virat
2,0,0,0,0,0,2,NaN,v Sri Lanka,Colombo (RPS),24 Aug 2008,ODI # 2750,Virat
3,0,0,0,0,0,2,NaN,v Sri Lanka,Colombo (RPS),27 Aug 2008,ODI # 2755,Virat
4,0,0,0,0,0,1,NaN,v Sri Lanka,Colombo (RPS),29 Aug 2008,ODI # 2756,Virat


In [569]:
df3.pop(df3.columns[10])

0       ODI # 2742
1       ODI # 2745
2       ODI # 2750
3       ODI # 2755
4       ODI # 2756
           ...    
1603    ODI # 4651
1604    ODI # 4654
1605    ODI # 4666
1606    ODI # 4669
1607    ODI # 4674
Name: Unnamed: 10, Length: 1608, dtype: object

In [570]:
df3.pop(df3.columns[6])

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
1603   NaN
1604   NaN
1605   NaN
1606   NaN
1607   NaN
Name: Unnamed: 6, Length: 1608, dtype: float64

In [571]:
df3.pop(df3.columns[1])

0       0
1       3
2       0
3       0
4       0
       ..
1603    0
1604    0
1605    1
1606    0
1607    0
Name: Ct, Length: 1608, dtype: object

In [572]:
df3.pop(df3.columns[0])

0       0
1       3
2       0
3       0
4       0
       ..
1603    0
1604    0
1605    1
1606    0
1607    0
Name: Dis, Length: 1608, dtype: object

In [573]:
df3.describe()

,St,Ct Wk,Ct Fi,Inns,Opposition,Ground,Start Date,Player
count,1608,1608,1608,1608,1608,1608,1608,1608
unique,3,5,5,3,15,73,373,17
top,0,0,0,1,v Australia,Colombo (RPS),14 Oct 2023,Virat
freq,1577,1553,1117,899,291,81,11,289


In [574]:
indian_stadiums = ['Eden Gardens', 'Wankhede', 'Lucknow', 'Dharamsala','Pune','Ahmedabad','Delhi','Chennai','Rajkot'\
'Visakhapatnam','Indore','Hyderabad','Thiruvananthapuram','Raipur','Guwahati','Bengaluru','Cuttack','Nagpur'\
                   ,'Ranchi','Mohali']
df3['Ground'] = df3['Ground'].apply(lambda x: 1 if x in indian_stadiums else 0)

In [575]:
df3.head()

,St,Ct Wk,Ct Fi,Inns,Opposition,Ground,Start Date,Player
0,0,0,0,2,v Sri Lanka,0,18 Aug 2008,Virat
1,0,0,3,1,v Sri Lanka,0,20 Aug 2008,Virat
2,0,0,0,2,v Sri Lanka,0,24 Aug 2008,Virat
3,0,0,0,2,v Sri Lanka,0,27 Aug 2008,Virat
4,0,0,0,1,v Sri Lanka,0,29 Aug 2008,Virat


In [576]:
df3['Start Date'] = pd.to_datetime(df3['Start Date'], format='%d %b %Y').dt.year

In [577]:
df3['St'] = pd.to_numeric(df3['St'], errors='coerce')
df3['Ct Wk'] = pd.to_numeric(df3['Ct Wk'], errors='coerce')
df3['Ct Fi'] = pd.to_numeric(df3['Ct Fi'], errors='coerce')

In [578]:
df3.isnull().sum()

St            24
Ct Wk         24
Ct Fi         24
Inns           0
Opposition     0
Ground         0
Start Date     0
Player         0
dtype: int64

In [579]:
df3.dropna(subset=['St'], inplace=True)

In [580]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(list(df3['Opposition']))
df3['Opposition'] = df3['Opposition'].apply(lambda x: le.transform([x])[0])

In [581]:
def wek(data,name):
  if data.shape[0] == 0:
    return 0
  beta_wicket_keeping_ability = 0.70*data['Ct Wk'].sum() + 0.30*data['St'].sum()
  wicket_keeper_score = 0.45 * count(df[df['Player']==name]) + 0.55*beta_wicket_keeping_ability
  return wicket_keeper_score

In [582]:
print(wek(df3[df3['Player']=='Rohit'],'Rohit'))

805.9480486666384


In [583]:
data = [{'Name': 'Virat', 'bat_score': 0,'bow_score': 0,'all_bat': 0,'all_bow': 0,'wkt': 0},
        {'Name': 'Rohit', 'bat_score': 0,'bow_score': 0,'all_bat': 0,'all_bow': 0,'wkt': 0},
        {'Name': 'Ishan', 'bat_score': 0,'bow_score': 0,'all_bat': 0,'all_bow': 0,'wkt': 0},
        {'Name': 'Sheryas', 'bat_score': 0,'bow_score': 0,'all_bat': 0,'all_bow': 0,'wkt': 0},
        {'Name': 'Rahul', 'bat_score': 0,'bow_score': 0,'all_bat': 0,'all_bow': 0,'wkt': 0},
        {'Name': 'Shubman', 'bat_score': 0,'bow_score': 0,'all_bat': 0,'all_bow': 0,'wkt': 0},
        {'Name': 'Hardik', 'bat_score': 0,'bow_score': 0,'all_bat': 0,'all_bow': 0,'wkt': 0},
        {'Name': 'Jasprit', 'bat_score': 0,'bow_score': 0,'all_bat': 0,'all_bow': 0,'wkt': 0},
        {'Name': 'Kuldeep', 'bat_score': 0,'bow_score': 0,'all_bat': 0,'all_bow': 0,'wkt': 0},
        {'Name': 'Shami', 'bat_score': 0,'bow_score': 0,'all_bat': 0,'all_bow': 0,'wkt': 0},
        {'Name': 'Axar', 'bat_score': 0,'bow_score': 0,'all_bat': 0,'all_bow': 0,'wkt': 0},
        {'Name': 'Shardul', 'bat_score': 0,'bow_score': 0,'all_bat': 0,'all_bow': 0,'wkt': 0},
        {'Name': 'Siraj', 'bat_score': 0,'bow_score': 0,'all_bat': 0,'all_bow': 0,'wkt': 0},
        {'Name': 'Suryakumar', 'bat_score': 0,'bow_score': 0,'all_bat': 0,'all_bow': 0,'wkt': 0},
        {'Name': 'Ashwin', 'bat_score': 0,'bow_score': 0,'all_bat': 0,'all_bow': 0,'wkt': 0},
        {'Name': 'Jadeja', 'bat_score': 0,'bow_score': 0,'all_bat': 0,'all_bow': 0,'wkt': 0},
        {'Name': 'Krishna', 'bat_score': 0,'bow_score': 0,'all_bat': 0,'all_bow': 0,'wkt': 0},]

final = pd.DataFrame(data)

In [584]:
final

,Name,bat_score,bow_score,all_bat,all_bow,wkt
0,Virat,0,0,0,0,0
1,Rohit,0,0,0,0,0
2,Ishan,0,0,0,0,0
3,Sheryas,0,0,0,0,0
4,Rahul,0,0,0,0,0
5,Shubman,0,0,0,0,0
6,Hardik,0,0,0,0,0
7,Jasprit,0,0,0,0,0
8,Kuldeep,0,0,0,0,0
9,Shami,0,0,0,0,0


In [595]:
for index, row in final.iterrows():
    name = 'U'
    for column in final.columns:
        if column == 'Name':
          name = final.at[index,column]
        elif column == 'bat_score':
          final.at[index,'bat_score'] = count(df[df['Player']==name])
        elif column == 'bow_score':
          final.at[index,'bow_score'] = count2(df2[df2['Player']==name])
        elif column == 'all_bat':
          final.at[index,'all_bat'] = batall(name)
        elif column == 'all_bow':
          final.at[index,'all_bow'] = bowall(name)
        elif column == 'wkt':
          final.at[index,'wkt'] = wek(df3[df3['Player']==name],name)


In [596]:
final

,Name,bat_score,bow_score,all_bat,all_bow,wkt
0,Virat,2354.867819,44.796571,1652.887131,1306.376443,1059.690519
1,Rohit,1790.995664,31.588151,1256.855780,992.944653,805.948049
2,Ishan,188.380333,0.000000,131.866233,103.609183,88.951150
3,Sheryas,401.829205,0.000000,281.280444,221.006063,180.823142
4,Rahul,471.210805,0.000000,329.847563,259.165943,227.499862
5,Shubman,412.826944,0.000000,288.978861,227.054819,185.772125
6,Hardik,331.249761,141.645038,246.039337,217.598628,149.062393
7,Jasprit,26.476739,196.011827,38.134900,63.565163,11.914532
8,Kuldeep,42.145158,221.036540,51.605265,78.438972,18.965321
9,Shami,48.247851,219.205300,55.694026,81.337643,21.711533
